In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import os
from pathlib import Path

print("Importing libraries successful!")
print(f"OpenCV version: {cv2.__version__}")

# Initialize YOLOv11 pose model
print("\nLoading YOLOv11 pose model...")
model = YOLO('yolo11n-pose.pt')  # Using YOLOv11 nano pose model
print("Model loaded successfully!")

Importing libraries successful!
OpenCV version: 4.12.0

Loading YOLOv11 pose model...
Model loaded successfully!
Model loaded successfully!


In [2]:
# Define video paths
VIDEO_DIR = Path(r"c:\Users\sapok\Documents\GitHub\MuayML")

videos = {
    'level_1': VIDEO_DIR / "Beginner_Karate.mp4",
    'level_2': VIDEO_DIR / "Semi_Advanced_Karate.mp4",
    'level_3': VIDEO_DIR / "Advanced_Karate.mp4",
    'noob': VIDEO_DIR / "Noob_Karate.mp4"
}

# Verify all videos exist
for name, path in videos.items():
    if path.exists():
        print(f"✓ Found {name}: {path.name}")
    else:
        print(f"✗ Missing {name}: {path.name}")

# YOLO Pose keypoint indices (COCO format)
# 0: nose, 1-2: eyes, 3-4: ears, 5: left shoulder, 6: right shoulder
# 7: left elbow, 8: right elbow, 9: left wrist, 10: right wrist
# 11: left hip, 12: right hip, 13: left knee, 14: right knee
# 15: left ankle, 16: right ankle

KEYPOINT_MAPPING = {
    'head': 0,  # nose
    'left_elbow': 7,
    'right_elbow': 8,
    'left_hand': 9,  # wrist
    'right_hand': 10,
    'left_hip': 11,
    'right_hip': 12,
    'chest': 5,  # approximating with left shoulder
    'left_knee': 13,
    'right_knee': 14,
    'left_toe': 15,  # ankle
    'right_toe': 16
}

print("\nKeypoint mapping defined successfully!")

✓ Found level_1: Beginner_Karate.mp4
✓ Found level_2: Semi_Advanced_Karate.mp4
✓ Found level_3: Advanced_Karate.mp4
✓ Found noob: Noob_Karate.mp4

Keypoint mapping defined successfully!


In [15]:
def extract_keypoints_from_frame(frame, model, confidence_threshold=0.5):
    """
    Extract keypoints from a single frame using YOLOv11 pose detection.
    Returns dictionary of keypoint coordinates with confidence filtering.
    """
    results = model(frame, verbose=False, conf=0.5)  # Increased confidence threshold
    
    if len(results) > 0 and results[0].keypoints is not None:
        keypoints = results[0].keypoints.xy.cpu().numpy()
        confidences = results[0].keypoints.conf.cpu().numpy()
        
        if len(keypoints) > 0 and len(confidences) > 0:
            # Get first person detected
            person_keypoints = keypoints[0]
            person_confidences = confidences[0]
            
            # Extract relevant keypoints with confidence filtering
            extracted = {}
            for name, idx in KEYPOINT_MAPPING.items():
                x, y = person_keypoints[idx]
                conf = person_confidences[idx]
                
                # Only include keypoint if detected with sufficient confidence
                if x > 0 and y > 0 and conf >= confidence_threshold:
                    extracted[name] = (int(x), int(y), float(conf))
                else:
                    extracted[name] = None
                    
            return extracted
    
    return None


def draw_keypoint_boxes(frame, keypoints, color=(0, 255, 0), box_size=20):
    """
    Draw precise boxes around detected keypoints with adaptive sizing.
    color: (B, G, R) format - (0, 255, 0) for GREEN, (0, 0, 255) for RED
    """
    # Define body-part specific box sizes for better precision
    body_part_sizes = {
        'head': 25,
        'left_elbow': 18,
        'right_elbow': 18,
        'left_hand': 15,
        'right_hand': 15,
        'left_hip': 20,
        'right_hip': 20,
        'chest': 22,
        'left_knee': 18,
        'right_knee': 18,
        'left_toe': 15,
        'right_toe': 15
    }
    
    for name, data in keypoints.items():
        if data is not None:
            x, y = data[0], data[1]
            conf = data[2] if len(data) > 2 else 1.0
            
            # Get body-part specific box size
            half_size = body_part_sizes.get(name, box_size) // 2
            
            # Draw rectangle with thickness based on confidence
            thickness = max(2, int(conf * 3))
            cv2.rectangle(frame, 
                         (x - half_size, y - half_size), 
                         (x + half_size, y + half_size), 
                         color, thickness)
            
            # Add label with confidence score
            label = f"{name.replace('_', ' ')}"
            
            # Position label above the box
            label_y = y - half_size - 8
            
            # Add background for text readability
            (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
            cv2.rectangle(frame, 
                         (x - half_size, label_y - text_height - 2),
                         (x - half_size + text_width + 2, label_y + 2),
                         color, -1)
            
            cv2.putText(frame, label, 
                       (x - half_size + 1, label_y),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 0), 1)
    
    return frame


print("Helper functions defined successfully!")

Helper functions defined successfully!


In [14]:
def process_reference_video(video_path, level_name, model, sample_frames=30):
    """
    Process reference videos (Beginner, Semi_Advanced, Advanced) to extract keypoints.
    Samples frames throughout the video to capture various poses.
    """
    print(f"\nProcessing {level_name} video: {video_path.name}")
    
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"Error: Could not open {video_path}")
        return None
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    print(f"  Total frames: {total_frames}, FPS: {fps}")
    
    # Sample frames evenly throughout the video
    frame_indices = np.linspace(0, total_frames - 1, sample_frames, dtype=int)
    
    all_keypoints = []
    
    for frame_idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        
        if not ret:
            continue
        
        keypoints = extract_keypoints_from_frame(frame, model)
        if keypoints:
            all_keypoints.append(keypoints)
    
    cap.release()
    
    print(f"  Extracted keypoints from {len(all_keypoints)} frames")
    
    return {
        'keypoints': all_keypoints,
        'fps': fps,
        'total_frames': total_frames
    }


# Store reference data for each level
reference_data = {}

print("Reference video processing function defined!")

Reference video processing function defined!


In [18]:
# Process all reference videos
print("="*60)
print("PROCESSING REFERENCE VIDEOS")
print("="*60)

reference_data['level_1'] = process_reference_video(videos['level_1'], 'Beginner (Level 1)', model)
reference_data['level_2'] = process_reference_video(videos['level_2'], 'Semi-Advanced (Level 2)', model)
reference_data['level_3'] = process_reference_video(videos['level_3'], 'Advanced (Level 3)', model)

print("\n" + "="*60)
print("REFERENCE DATA EXTRACTION COMPLETE")
print("="*60)

PROCESSING REFERENCE VIDEOS

Processing Beginner (Level 1) video: Beginner_Karate.mp4
  Total frames: 251, FPS: 25.0
  Extracted keypoints from 30 frames

Processing Semi-Advanced (Level 2) video: Semi_Advanced_Karate.mp4
  Total frames: 420, FPS: 25.0
  Extracted keypoints from 30 frames

Processing Semi-Advanced (Level 2) video: Semi_Advanced_Karate.mp4
  Total frames: 420, FPS: 25.0
  Extracted keypoints from 30 frames

Processing Advanced (Level 3) video: Advanced_Karate.mp4
  Total frames: 555, FPS: 29.97002997002997
  Extracted keypoints from 30 frames

Processing Advanced (Level 3) video: Advanced_Karate.mp4
  Total frames: 555, FPS: 29.97002997002997
  Extracted keypoints from 30 frames

REFERENCE DATA EXTRACTION COMPLETE
  Extracted keypoints from 30 frames

REFERENCE DATA EXTRACTION COMPLETE


In [16]:
def calculate_keypoint_distance(kp1, kp2):
    """Calculate Euclidean distance between two keypoints."""
    if kp1 is None or kp2 is None:
        return float('inf')
    
    # Extract x, y coordinates (ignoring confidence if present)
    x1, y1 = kp1[0], kp1[1]
    x2, y2 = kp2[0], kp2[1]
    
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)


def compare_poses_with_level1(noob_keypoints, level1_keypoints_list):
    """
    Compare noob keypoints ONLY with Level 1 (Beginner) reference keypoints.
    Returns: dict with match status for each keypoint and overall match percentage.
    """
    if noob_keypoints is None or not level1_keypoints_list:
        return None
    
    keypoint_matches = {}
    match_count = 0
    total_count = 0
    
    # Body-part specific thresholds for more accurate matching
    body_part_thresholds = {
        'head': 60,
        'left_elbow': 80,
        'right_elbow': 80,
        'left_hand': 100,
        'right_hand': 100,
        'left_hip': 70,
        'right_hip': 70,
        'chest': 60,
        'left_knee': 80,
        'right_knee': 80,
        'left_toe': 100,
        'right_toe': 100
    }
    
    # For each keypoint in the noob's pose
    for keypoint_name in KEYPOINT_MAPPING.keys():
        noob_kp = noob_keypoints.get(keypoint_name)
        
        if noob_kp is None:
            keypoint_matches[keypoint_name] = {'matched': False, 'distance': float('inf'), 'confidence': 0}
            total_count += 1
            continue
        
        # Get confidence if available
        conf = noob_kp[2] if len(noob_kp) > 2 else 1.0
        
        # Find the best match from Level 1 reference frames ONLY
        best_distance = float('inf')
        
        for ref_keypoints in level1_keypoints_list:
            ref_kp = ref_keypoints.get(keypoint_name)
            distance = calculate_keypoint_distance(noob_kp, ref_kp)
            
            if distance < best_distance:
                best_distance = distance
        
        # Use body-part specific threshold
        specific_threshold = body_part_thresholds.get(keypoint_name, 100)
        
        # Determine if this keypoint matches (considering both distance and confidence)
        matched = best_distance < specific_threshold and conf >= 0.5
        
        keypoint_matches[keypoint_name] = {
            'matched': matched,
            'distance': best_distance,
            'confidence': conf
        }
        
        if matched:
            match_count += 1
        total_count += 1
    
    match_percentage = (match_count / total_count * 100) if total_count > 0 else 0
    
    return {
        'keypoint_matches': keypoint_matches,
        'match_percentage': match_percentage,
        'matched_count': match_count,
        'total_count': total_count
    }


print("Pose comparison functions defined successfully!")

Pose comparison functions defined successfully!


In [19]:
def create_level1_reference_video(video_path, model, output_path):
    """
    Create Level 1 reference video with ALL GREEN boxes around detected keypoints.
    """
    print("\n" + "="*60)
    print("CREATING LEVEL 1 REFERENCE VIDEO")
    print("="*60)
    
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"Error: Could not open {video_path}")
        return None
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"\nVideo properties:")
    print(f"  Resolution: {width}x{height}")
    print(f"  FPS: {fps}")
    print(f"  Total frames: {total_frames}")
    
    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))
    
    frame_count = 0
    
    # Define body-part specific box sizes
    body_part_sizes = {
        'head': 25,
        'left_elbow': 18,
        'right_elbow': 18,
        'left_hand': 15,
        'right_hand': 15,
        'left_hip': 20,
        'right_hip': 20,
        'chest': 22,
        'left_knee': 18,
        'right_knee': 18,
        'left_toe': 15,
        'right_toe': 15
    }
    
    print("\nProcessing frames...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Extract keypoints
        keypoints = extract_keypoints_from_frame(frame, model, confidence_threshold=0.6)
        
        if keypoints:
            # Draw ALL boxes in GREEN
            for keypoint_name, data in keypoints.items():
                if data is not None:
                    x, y = data[0], data[1]
                    conf = data[2] if len(data) > 2 else 1.0
                    
                    # ALL GREEN for reference video
                    color = (0, 255, 0)  # GREEN
                    
                    # Get body-part specific box size
                    half_size = body_part_sizes.get(keypoint_name, 20) // 2
                    
                    # Draw rectangle
                    thickness = 3
                    cv2.rectangle(frame, 
                                 (x - half_size, y - half_size), 
                                 (x + half_size, y + half_size), 
                                 color, thickness)
                    
                    # Add label
                    label = keypoint_name.replace('_', ' ')
                    label_y = y - half_size - 8
                    
                    # Add semi-transparent background for text
                    (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)
                    
                    overlay = frame.copy()
                    cv2.rectangle(overlay, 
                                 (x - half_size - 2, label_y - text_height - 3),
                                 (x - half_size + text_width + 2, label_y + 3),
                                 color, -1)
                    cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
                    
                    cv2.putText(frame, label, 
                               (x - half_size, label_y),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Add title overlay
            overlay = frame.copy()
            cv2.rectangle(overlay, (0, 0), (width, 60), (0, 0, 0), -1)
            cv2.addWeighted(overlay, 0.5, frame, 0.5, 0, frame)
            
            cv2.putText(frame, "LEVEL 1 - BEGINNER REFERENCE", 
                       (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Write frame
        out.write(frame)
        
        if frame_count % 30 == 0:
            print(f"  Processed {frame_count}/{total_frames} frames...")
    
    cap.release()
    out.release()
    
    print(f"\n✓ Level 1 reference video saved to: {output_path}")
    print(f"✓ Processed {frame_count} frames")


def process_and_analyze_noob_video(noob_video_path, level1_reference_data, model, output_path):
    """
    Process the Noob_Karate video, compare ONLY with Level 1 reference,
    and create an annotated output video with GREEN for correct and RED for incorrect.
    """
    print("\n" + "="*60)
    print("PROCESSING NOOB VIDEO")
    print("="*60)
    
    cap = cv2.VideoCapture(str(noob_video_path))
    if not cap.isOpened():
        print(f"Error: Could not open {noob_video_path}")
        return None
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"\nVideo properties:")
    print(f"  Resolution: {width}x{height}")
    print(f"  FPS: {fps}")
    print(f"  Total frames: {total_frames}")
    print(f"\n🎯 Comparing ONLY with Level 1 (Beginner) reference")
    
    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))
    
    frame_results = []
    frame_count = 0
    
    print("\nProcessing frames...")
    
    # Define body-part specific box sizes
    body_part_sizes = {
        'head': 25,
        'left_elbow': 18,
        'right_elbow': 18,
        'left_hand': 15,
        'right_hand': 15,
        'left_hip': 20,
        'right_hip': 20,
        'chest': 22,
        'left_knee': 18,
        'right_knee': 18,
        'left_toe': 15,
        'right_toe': 15
    }
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Extract keypoints from current frame
        noob_keypoints = extract_keypoints_from_frame(frame, model, confidence_threshold=0.6)
        
        if noob_keypoints:
            # Compare ONLY with Level 1
            comparison = compare_poses_with_level1(noob_keypoints, level1_reference_data['keypoints'])
            
            if comparison:
                # Draw boxes based on match status
                for keypoint_name, data in noob_keypoints.items():
                    if data is not None:
                        x, y = data[0], data[1]
                        conf = data[2] if len(data) > 2 else 1.0
                        
                        match_info = comparison['keypoint_matches'].get(keypoint_name, {})
                        
                        # GREEN for matched, RED for not matched
                        if match_info.get('matched', False):
                            color = (0, 255, 0)  # GREEN - CORRECT
                        else:
                            color = (0, 0, 255)  # RED - WRONG / DEVIATION
                        
                        # Get body-part specific box size
                        half_size = body_part_sizes.get(keypoint_name, 20) // 2
                        
                        # Draw rectangle - thicker and brighter RED for wrong positions
                        thickness = 3 if match_info.get('matched', False) else 4
                        cv2.rectangle(frame, 
                                     (x - half_size, y - half_size), 
                                     (x + half_size, y + half_size), 
                                     color, thickness)
                        
                        # Add label with clear indication
                        label = keypoint_name.replace('_', ' ')
                        if not match_info.get('matched', False):
                            label += " ✗ WRONG"
                        
                        # Position label above the box
                        label_y = y - half_size - 8
                        
                        # Add semi-transparent background for text
                        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)
                        
                        overlay = frame.copy()
                        cv2.rectangle(overlay, 
                                     (x - half_size - 2, label_y - text_height - 3),
                                     (x - half_size + text_width + 2, label_y + 3),
                                     color, -1)
                        cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
                        
                        cv2.putText(frame, label, 
                                   (x - half_size, label_y),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1, cv2.LINE_AA)
                
                # Add info overlay
                match_pct = comparison['match_percentage']
                matched = comparison['matched_count']
                total = comparison['total_count']
                
                # Create info panel at top
                overlay = frame.copy()
                cv2.rectangle(overlay, (0, 0), (width, 90), (0, 0, 0), -1)
                cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
                
                # Color code the match percentage
                if match_pct >= 70:
                    perf_color = (0, 255, 0)  # GREEN
                elif match_pct >= 50:
                    perf_color = (0, 255, 255)  # YELLOW
                else:
                    perf_color = (0, 0, 255)  # RED
                
                cv2.putText(frame, f"Match with Level 1: {match_pct:.1f}% ({matched}/{total} correct)", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, perf_color, 2, cv2.LINE_AA)
                
                cv2.putText(frame, f"Frame: {frame_count}/{total_frames}", 
                           (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1, cv2.LINE_AA)
                
                frame_results.append({
                    'frame': frame_count,
                    'match_percentage': match_pct,
                    'comparison': comparison
                })
        
        # Write frame to output
        out.write(frame)
        
        if frame_count % 30 == 0:
            print(f"  Processed {frame_count}/{total_frames} frames...")
    
    cap.release()
    out.release()
    
    print(f"\n✓ Output video saved to: {output_path}")
    print(f"✓ Processed {frame_count} frames")
    
    return frame_results


# Create Level 1 reference video with all GREEN boxes
print("\n" + "="*60)
print("STEP 1: Creating Level 1 Reference Video")
print("="*60)
level1_output_path = VIDEO_DIR / "level1_reference_analyzed.mp4"
create_level1_reference_video(videos['level_1'], model, level1_output_path)

# Process the Noob video comparing ONLY with Level 1
print("\n" + "="*60)
print("STEP 2: Analyzing Noob Video against Level 1")
print("="*60)
noob_output_path = VIDEO_DIR / "karate_analyzed.mp4"
noob_results = process_and_analyze_noob_video(videos['noob'], reference_data['level_1'], model, noob_output_path)

print("\n" + "="*60)
print("VIDEO PROCESSING COMPLETE")
print("="*60)


STEP 1: Creating Level 1 Reference Video

CREATING LEVEL 1 REFERENCE VIDEO

Video properties:
  Resolution: 1080x1920
  FPS: 25.0
  Total frames: 251

Processing frames...
  Processed 30/251 frames...
  Processed 30/251 frames...
  Processed 60/251 frames...
  Processed 60/251 frames...
  Processed 90/251 frames...
  Processed 90/251 frames...
  Processed 120/251 frames...
  Processed 120/251 frames...
  Processed 150/251 frames...
  Processed 150/251 frames...
  Processed 180/251 frames...
  Processed 180/251 frames...
  Processed 210/251 frames...
  Processed 210/251 frames...
  Processed 240/251 frames...
  Processed 240/251 frames...

✓ Level 1 reference video saved to: c:\Users\sapok\Documents\GitHub\MuayML\level1_reference_analyzed.mp4
✓ Processed 251 frames

STEP 2: Analyzing Noob Video against Level 1

PROCESSING NOOB VIDEO

Video properties:
  Resolution: 1080x1920
  FPS: 25.0
  Total frames: 270

🎯 Comparing ONLY with Level 1 (Beginner) reference

Processing frames...

✓ Lev

In [20]:
# Generate comprehensive analysis and grading
print("\n" + "="*60)
print("GENERATING ANALYSIS REPORT")
print("="*60)

if noob_results:
    # Calculate overall statistics
    total_frames_analyzed = len(noob_results)
    
    if total_frames_analyzed > 0:
        avg_match_percentage = np.mean([r['match_percentage'] for r in noob_results])
        
        # Calculate grade (1-100) based on match with Level 1
        grade = int(avg_match_percentage)
        
        # Generate detailed report
        print("\n📊 PERFORMANCE ANALYSIS (vs Level 1 Beginner)")
        print("-" * 60)
        print(f"Total Frames Analyzed: {total_frames_analyzed}")
        print(f"Average Match Percentage: {avg_match_percentage:.2f}%")
        print(f"\n⭐ Overall Grade: {grade}/100")
        
        # Performance assessment
        print("\n💬 ASSESSMENT")
        print("-" * 60)
        if grade >= 90:
            assessment = "EXCELLENT! Your karate form closely matches the Level 1 beginner techniques."
            emoji = "🏆"
        elif grade >= 75:
            assessment = "GOOD! Your form is solid with some minor deviations from Level 1."
            emoji = "✅"
        elif grade >= 60:
            assessment = "FAIR. Your form shows promise but needs improvement in several areas."
            emoji = "⚠️"
        elif grade >= 40:
            assessment = "NEEDS WORK. Significant differences from Level 1 techniques detected."
            emoji = "❌"
        else:
            assessment = "BEGINNER. Focus on mastering basic Level 1 form and positioning."
            emoji = "📚"
        
        print(f"{emoji} {assessment}")
        
        # Keypoint-specific analysis
        print("\n🎯 KEYPOINT ANALYSIS (Body Part Performance)")
        print("-" * 60)
        
        # Aggregate keypoint match rates
        keypoint_stats = {kp: {'matched': 0, 'total': 0, 'avg_distance': []} for kp in KEYPOINT_MAPPING.keys()}
        
        for result in noob_results:
            for kp_name, match_info in result['comparison']['keypoint_matches'].items():
                keypoint_stats[kp_name]['total'] += 1
                if match_info['matched']:
                    keypoint_stats[kp_name]['matched'] += 1
                if match_info['distance'] != float('inf'):
                    keypoint_stats[kp_name]['avg_distance'].append(match_info['distance'])
        
        print("\nMatch rates by body part:")
        print(f"{'Body Part':<18} {'Match Rate':<12} {'Status':<10} {'Avg Distance (px)'}")
        print("-" * 60)
        
        for kp_name, stats in keypoint_stats.items():
            if stats['total'] > 0:
                match_rate = (stats['matched'] / stats['total']) * 100
                avg_dist = np.mean(stats['avg_distance']) if stats['avg_distance'] else 0
                
                if match_rate >= 80:
                    status = "✓ GOOD"
                elif match_rate >= 60:
                    status = "⚠ OK"
                else:
                    status = "✗ POOR"
                
                print(f"{kp_name:<18} {match_rate:>5.1f}% ({stats['matched']:>3}/{stats['total']:<3}) {status:<10} {avg_dist:>6.1f}")
        
        # Recommendations
        print("\n📋 RECOMMENDATIONS")
        print("-" * 60)
        poor_keypoints = [kp for kp, stats in keypoint_stats.items() 
                         if stats['total'] > 0 and (stats['matched'] / stats['total']) < 0.6]
        
        if poor_keypoints:
            print("Focus on improving these body parts:")
            for kp in poor_keypoints[:5]:  # Show top 5
                print(f"  • {kp.replace('_', ' ').title()}")
        else:
            print("Great job! All body parts show good form.")
        
        print("\n" + "="*60)
        print(f"📹 OUTPUT VIDEOS:")
        print(f"   1. Level 1 Reference: level1_reference_analyzed.mp4")
        print(f"   2. Noob Analysis: karate_analyzed.mp4")
        print("="*60)
        
        print("\n✅ ANALYSIS COMPLETE!")
        print(f"\n🎯 Summary: You scored {grade}/100 compared to Level 1 Beginner techniques.")
        
else:
    print("⚠️ No results to analyze. Check if the Noob video was processed correctly.")


GENERATING ANALYSIS REPORT

📊 PERFORMANCE ANALYSIS (vs Level 1 Beginner)
------------------------------------------------------------
Total Frames Analyzed: 270
Average Match Percentage: 87.31%

⭐ Overall Grade: 87/100

💬 ASSESSMENT
------------------------------------------------------------
✅ GOOD! Your form is solid with some minor deviations from Level 1.

🎯 KEYPOINT ANALYSIS (Body Part Performance)
------------------------------------------------------------

Match rates by body part:
Body Part          Match Rate   Status     Avg Distance (px)
------------------------------------------------------------
head               100.0% (270/270) ✓ GOOD       22.5
left_elbow          81.1% (219/270) ✓ GOOD       52.0
right_elbow         96.3% (260/270) ✓ GOOD       28.0
left_hand           62.2% (168/270) ⚠ OK         91.7
right_hand          99.6% (269/270) ✓ GOOD       20.7
left_hip            99.3% (268/270) ✓ GOOD       27.0
right_hip           98.5% (266/270) ✓ GOOD       21.9
ches